
#### Acessando o Storage da Azure 

###### Por motivos de problemas com o Unity Catalog, depois de pesquisas achei a alternativa de Montar com "dbutils" e até o momento está funcional

In [0]:
# Acesso do Blob Storage
container_name = "lake"
storage_account_name = "lakesavana"
mount_point = "/mnt/savana"
access_key = "M4OYEhV7y/mOowqVqZRcLfm2taCeU7eF8WEROHP5gEV22EbpDFUZx5OsCtzjeoyJ2fna0FRGjqaz+AStYvlGeA=="

# Configurar acesso
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    access_key
)

# Montar (executar só uma vez — se já montou, ignore)
dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point = mount_point,
    extra_configs = {
        f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key
    }
)



#### Leitura dos Arquivos "Parquet"

###### Agora vou Listar o que tem Dentro do Storage, ler o arquivo como Dataframe para ver como estão os vindo os Dados

In [0]:
# Listar os Arquivos do Blob (Finalidade é para confirmar que está funcionando a Conexão)
dbutils.fs.ls("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/")


In [0]:
# Lê cada pasta
df_1 = spark.read.format("delta").load("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/0a6c3f04-3748-4d9e-9b2f-6400c0dc680b")
df_2 = spark.read.format("delta").load("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/2e95a1bd-6dcb-4bdb-84db-ba3925c1dff8")
df_3 = spark.read.format("delta").load("/mnt/savana/__unitystorage/catalogs/10af6d1d-32a1-46c7-95d7-ef4fbde791d4/tables/290d3ddb-6031-495f-a097-fbb09f50f4d8")

# Exibir os dados
df_1.display()
df_2.display()
df_3.display()



#### Efetuar a Camada Bronze

###### Agora vou pegar os Dataframes que Listei acima e transforma-los em Tabela

In [0]:
# Tabelas Bronze no catálogo (Dataframe 2 está vindo vazio, por agora irei desconsidera-lo)
df_1.write.mode("overwrite").saveAsTable("savana.daniel.bronze.transacoes")
df_3.write.mode("overwrite").saveAsTable("savana.daniel.bronze.clientes")
